In [ ]:
# 📘 Sahayak: Complete Agent Orchestration Testing

import sys, os
sys.path.append(os.path.abspath(".."))

# ✅ Initialize Environment
from utils.setup_env import configure_environment
configure_environment()


In [ ]:

from agents.agent_manager import AgentManager
from agents.agent_router import AgentRouter, AgentType
import json, time

# ⚙️ Globals
agent_manager = None
routing_results = []
execution_results = []

# 🚀 Initialize Agent Orchestration
print("🚀 Initializing Sahayak Agent Orchestration System...\n")

try:
    agent_manager = AgentManager()
    print("✅ Agent Manager initialized successfully\n")

    available_agents = agent_manager.list_available_agents()
    print(f"📋 Available Agents ({len(available_agents)}):")
    for name, desc in available_agents.items():
        print(f"  • {name}: {desc}")

except Exception as e:
    import traceback
    print(f"❌ Error initializing Agent Manager: {e}")
    traceback.print_exc()

# 🧪 Begin Testing if initialized
if agent_manager is None:
    raise RuntimeError("❌ AgentManager failed to initialize. Cannot proceed with tests.")

print("\n" + "="*60)
print("🧪 TESTING AGENT ROUTING AND ORCHESTRATION")
print("="*60)

# ✅ Test Cases for Routing and Execution
test_requests = [
    {
        "request": "Why is the sky blue? Explain in Hindi for grade 3 students",
        "expected_agent": AgentType.DOUBT_ASSISTANT,
        "description": "Basic explanation request"
    },
    {
        "request": "Create a story about a farmer in Marathi for grade 5 students learning about soil types",
        "expected_agent": AgentType.CONTENT_GENERATION,
        "description": "Story generation"
    },
    {
        "request": "Draw a simple diagram showing water cycle that I can recreate on blackboard",
        "expected_agent": AgentType.DRAWINGS_AGENT,
        "description": "Visual aid request"
    },
    {
        "request": "Create a concept map organizing all topics in grade 6 history chapter on Mughal Empire",
        "expected_agent": AgentType.MINDMAP_AGENT,
        "description": "Concept map"
    },
    {
        "request": "Help me with teaching",
        "expected_agent": AgentType.DOUBT_ASSISTANT,
        "description": "Ambiguous request (should fallback)"
    },
    {
        "request": "I have uploaded a photo of textbook page, create worksheets for different grade levels",
        "expected_agent": AgentType.VISION_AGENT,
        "description": "Image processing and worksheet generation",
        "context": {
            "task_type": "extract_text",
            "image_path": "../data/images/biology_textbook.png"
        },
        "follow_up": {
            "task_type": "generate_worksheets",
            "target_grades": [3, 5]  # or [9, 10] based on your use case
        }
    },

    {
        "request": "Plan weekly lessons for teaching science to mixed grades 3-5 in rural school",
        "expected_agent": AgentType.LESSON_PLANNER,
        "description": "Multi-grade lesson planning",
        "context": {
            "task_type": "weekly",
            "subjects": ["science"],
            "grade_levels": [3, 4],
            "language": "english",
            "total_hours": 30
        }
    }

]

print(f"\n🔍 Running {len(test_requests)} test requests...\n")

router = AgentRouter()
routing_accuracy = 0

for i, case in enumerate(test_requests, 1):
    print(f"🧪 Test {i}: {case['description']}")
    print(f"Request: {case['request']}")
    
    try:
        # 🔄 Routing
        route = router.route_request(case['request'])
        expected = case['expected_agent'].value
        actual = route.agent_type.value
        correct = (expected == actual)

        # 🧠 Print Routing
        print(f"🎯 Routed to: {actual} | ✅ Correct: {correct}")
        print(f"🎲 Confidence: {route.confidence:.2f}")
        print(f"📝 Reasoning: {route.reasoning}")

        routing_results.append({
            'test_case': i,
            'request': case['request'],
            'expected': expected,
            'actual': actual,
            'correct': correct,
            'confidence': route.confidence,
            'reasoning': route.reasoning
        })
        if correct: routing_accuracy += 1

        # 🚀 Execute Task
        print("⚙️ Executing full request via AgentManager...")
        start = time.time()
        
        response = agent_manager.process_request(
            case['request'],
            context=case.get('context', {})
        )

        duration = time.time() - start

        print(f"⏱️ Time: {duration:.2f}s | ✅ Success: {response.success}")

        if response.success:
            print(f"🤖 Agent Used: {response.agent_name}")
            print(f"📦 Response Preview: {str(response.data)[:200]}...\n")
        else:
            print(f"❌ Error: {response.error}\n")

        execution_results.append({
            'test_case': i,
            'success': response.success,
            'execution_time': duration,
            'agent_used': response.agent_name,
            'error': response.error
        })

    except Exception as e:
        import traceback
        print(f"❌ Exception during test {i}: {e}")
        traceback.print_exc()
    
    print("-" * 60)

# 📊 Summary
print("\n📊 TEST SUMMARY")
print("="*60)

total_tests = len(test_requests)
success_count = sum(1 for r in routing_results if r['correct'])
execution_success = sum(1 for r in execution_results if r['success'])

print(f"🎯 Routing Accuracy: {success_count}/{total_tests} = {100*success_count/total_tests:.1f}%")
print(f"⚡ Execution Success: {execution_success}/{total_tests} = {100*execution_success/total_tests:.1f}%")

# ✅ Health Check
print("\n🔍 SYSTEM HEALTH CHECK")
print("="*60)

health = agent_manager.health_check()
print(f"🏥 System Status: {health['system_status'].upper()}")

for name, status in health['agent_status'].items():
    icon = "✅" if status == "healthy" else "⚠️" if status == "unknown" else "❌"
    print(f"{icon} {name}: {status}")

# 📈 Agent Statistics
print("\n📈 AGENT STATISTICS")
print("="*60)

stats = agent_manager.get_agent_stats()
for agent, s in stats['agent_statistics'].items():
    if s['total_requests'] > 0:
        print(f"• {agent}: {s['total_requests']} requests, {s['successful_requests']} success, Avg time: {s['avg_response_time']:.2f}s")
